In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [2]:
spark = SparkSession\
        .builder\
        .appName('bci')\
        .master('local')\
        .getOrCreate()

23/12/10 11:40:03 WARN Utils: Your hostname, MyBeast resolves to a loopback address: 127.0.1.1; using 172.18.210.214 instead (on interface eth0)
23/12/10 11:40:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/10 11:40:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load and transform the data

In [3]:
df = spark.read\
    .options(header=True,inferSchema='True',delimiter=',')\
    .csv("data/sub3_comp.csv")

In [4]:
windowSpec = Window.orderBy(F.lit(1))

In [5]:
df = df.withColumn("ms", F.row_number().over(windowSpec))

In [6]:
df = df.withColumn("step", F.ceil(df['ms']/100))

In [7]:
df.select("ms","step","thumb").show(200)

23/12/10 11:41:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/10 11:41:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/10 11:41:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/10 11:41:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/10 11:41:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+----+--------------------+
| ms|step|               thumb|
+---+----+--------------------+
|  1|   1|-0.41384039298833086|
|  2|   1|-0.41384039298833086|
|  3|   1|-0.41384039298833086|
|  4|   1|-0.41384039298833086|
|  5|   1|-0.41384039298833086|
|  6|   1|-0.41384039298833086|
|  7|   1|-0.41384039298833086|
|  8|   1|-0.41384039298833086|
|  9|   1|-0.41384039298833086|
| 10|   1|-0.41384039298833086|
| 11|   1|-0.41384039298833086|
| 12|   1|-0.41384039298833086|
| 13|   1|-0.41384039298833086|
| 14|   1|-0.41384039298833086|
| 15|   1|-0.41384039298833086|
| 16|   1|-0.41384039298833086|
| 17|   1|-0.41384039298833086|
| 18|   1|-0.41384039298833086|
| 19|   1|-0.41384039298833086|
| 20|   1|-0.41384039298833086|
| 21|   1|-0.41384039298833086|
| 22|   1|-0.41384039298833086|
| 23|   1|-0.41384039298833086|
| 24|   1|-0.41384039298833086|
| 25|   1|-0.41384039298833086|
| 26|   1|-0.41384039298833086|
| 27|   1|-0.41384039298833086|
| 28|   1|-0.41384039298833086|
| 29|   

In [13]:
num_files = 4000


In [14]:
df = df.repartition(num_files)

df.write.csv("data/processed_bci_csv",
             mode = "overwrite",
             header = True)

23/12/10 11:45:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/10 11:45:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/10 11:45:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/12/10 11:45:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/10 11:45:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


## Get Data Schema

In [15]:
dataSchema = df.schema

## Set up stream

In [49]:
signals = spark\
            .readStream\
            .format('csv')\
            .schema(dataSchema)\
            .option("header",True)\
            .option("maxFilesPerTrigger",1)\
            .load(r"data/processed_bci_csv/")

In [51]:
signals = signals.withWatermark(
    "step",
    "1 second"
)

AnalysisException: [EVENT_TIME_IS_NOT_ON_TIMESTAMP_TYPE] The event time `step` has the invalid type "BIGINT", but expected "TIMESTAMP".;
EventTimeWatermark step#120548: bigint, 1 seconds
+- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@279e4b7c,csv,List(),Some(StructType(StructField(thumb,DoubleType,true),StructField(index,DoubleType,true),StructField(middle,DoubleType,true),StructField(ring,DoubleType,true),StructField(little,DoubleType,true),StructField(channel_1,DoubleType,true),StructField(channel_2,DoubleType,true),StructField(channel_3,DoubleType,true),StructField(channel_4,DoubleType,true),StructField(channel_5,DoubleType,true),StructField(channel_6,DoubleType,true),StructField(channel_7,DoubleType,true),StructField(channel_8,DoubleType,true),StructField(channel_9,DoubleType,true),StructField(channel_10,DoubleType,true),StructField(channel_11,DoubleType,true),StructField(channel_12,DoubleType,true),StructField(channel_13,DoubleType,true),StructField(channel_14,D

In [18]:
signals.isStreaming

True

In [19]:
signals.printSchema()

root
 |-- thumb: double (nullable = true)
 |-- index: double (nullable = true)
 |-- middle: double (nullable = true)
 |-- ring: double (nullable = true)
 |-- little: double (nullable = true)
 |-- channel_1: double (nullable = true)
 |-- channel_2: double (nullable = true)
 |-- channel_3: double (nullable = true)
 |-- channel_4: double (nullable = true)
 |-- channel_5: double (nullable = true)
 |-- channel_6: double (nullable = true)
 |-- channel_7: double (nullable = true)
 |-- channel_8: double (nullable = true)
 |-- channel_9: double (nullable = true)
 |-- channel_10: double (nullable = true)
 |-- channel_11: double (nullable = true)
 |-- channel_12: double (nullable = true)
 |-- channel_13: double (nullable = true)
 |-- channel_14: double (nullable = true)
 |-- channel_15: double (nullable = true)
 |-- channel_16: double (nullable = true)
 |-- channel_17: double (nullable = true)
 |-- channel_18: double (nullable = true)
 |-- channel_19: double (nullable = true)
 |-- channel_20: dou

## Create transformation

In [37]:
average_readings = signals.select([F.avg(col).alias('avg_' + col) for col in ['thumb','index', 'middle', 'ring']])

In [40]:
import requests

In [46]:
def send_batch_to_http_server(batch_df, epoch_id):
    # Convert the DataFrame to a JSON string or another suitable format
    json_data = batch_df.toJSON().collect()
    
    # HTTP POST request for each record in the batch
    for record in json_data:
        response = requests.post('ttp://127.0.0.1:5000/post_data', json=record)
        # Handle the response if necessary

In [53]:
query = average_readings.writeStream\
        .outputMode("update")\
        .foreachBatch(send_batch_to_http_server)\
        .trigger(processingTime='1000 milliseconds')\
        .start()

23/12/10 12:29:26 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-65549593-8344-4ad9-a349-0875b47ed624. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/10 12:29:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/12/10 12:29:27 ERROR MicroBatchExecution: Query [id = af282eae-4797-4640-ab35-1c0cd9b1c388, runId = 31145af9-822b-44b8-bbdb-8de9537b611c] terminated with error
py4j.Py4JException: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/home/hdoop/bci/bci/lib/python3.10/site-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/home/hdoop/bci/

In [45]:
query.stop()